In [28]:
import pandas as pd
import numpy as np
from forest_reg import MyForestReg  # Импорт дерева
from sklearn.datasets import make_regression
import random

In [ ]:
class MyBoostReg:
    def __init__(
        self,
        n_estimators=10,
        max_depth=5,
        min_samples_split=2,
        max_leafs=20,
        bins=None,
        loss="MSE",  # Добавлен параметр loss
        metric=None,  # Метрика для оценки модели
        max_features=0.5, #доля фичей
        max_samples=0.5, #доля семплом
        random_state=42,
        learning_rate=0.1,
        reg=0.1  # Новый параметр регуляризации
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.bins = bins
        self.pred_0 = None  # Изначальное предсказание (среднее по таргету)
        self.trees = []     # Список для хранения обученных деревьев
        self.loss = loss
        self.metric = metric
        self.best_score = None  # Хранение лучшего значения метрики
        self.max_features = max_features
        self.max_samples = max_samples
        self.random_state = random_state
        self.learning_rate = learning_rate  # Может быть числом или функцией
        self.reg = reg  # Регуляризация
        # Важность фичей
        self.fi = {}  # Словарь для хранения важности признаков

    def __str__(self):
        params = vars(self)  # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyForestReg class: {params_str}"
    
    # Метод для расчёта функции потерь
    def _calculate_loss(self, y_true, y_pred):
        if self.loss == "MSE":
            return np.mean((y_true - y_pred) ** 2)
        elif self.loss == "MAE":
            return np.mean(np.abs(y_true - y_pred))
        else:
            raise ValueError("Invalid loss function. Supported: 'MSE', 'MAE'.")
        
    # Расчет метрик
    def _calculate_metric(self, y_true, y_pred):
        if self.metric == "MSE":
            return np.mean((y_true - y_pred) ** 2)
        elif self.metric == "MAE":
            return np.mean(np.abs(y_true - y_pred))
        elif self.metric == "RMSE":
            return np.sqrt(np.mean((y_true - y_pred) ** 2))
        elif self.metric == "R2":
            ss_total = np.sum((y_true - y_true.mean()) ** 2)
            ss_residual = np.sum((y_true - y_pred) ** 2)
            return 1 - (ss_residual / ss_total)
        elif self.metric == "MAPE":
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        else:
            raise ValueError("Invalid metric specified.")
    
  # Метод fit
    def fit(self, X: pd.DataFrame, y: pd.Series, X_eval=None, y_eval=None, early_stopping=None, verbose=None):
        # Фиксируем сид
        random.seed(self.random_state)
        
        self.pred_0 = y.mean()
        current_prediction = np.full(y.shape, self.pred_0)
        
         # Инициализируем важность признаков
        self.fi = {col: 0.0 for col in X.columns}

        init_cols = list(X.columns)  # Все признаки
        init_rows_cnt = X.shape[0]  # Общее число строк
        
        # добавляем параметры отслеживания остановки
        best_score = None
        best_iter = 0

        for i in range(1, self.n_estimators + 1): # Нумерация итераций от 1 до n_estimators
            # Вычисляем learning_rate для текущего шага
            if callable(self.learning_rate):
                lr = self.learning_rate(i)
            else:
                lr = self.learning_rate
            
            # Случайный выбор признаков и строк
            cols_smpl_cnt = max(1, round(self.max_features * len(init_cols)))
            rows_smpl_cnt = max(1, round(self.max_samples * init_rows_cnt))

            cols_idx = random.sample(init_cols, cols_smpl_cnt)
            rows_idx = random.sample(range(init_rows_cnt), rows_smpl_cnt)

            X_sample = X.iloc[rows_idx][cols_idx]
            y_sample = y.iloc[rows_idx]

            # Остатки
            residual = y_sample - current_prediction[rows_idx]

            # Обучение дерева на подвыборке
            tree = self._build_tree(X_sample, residual)
            num_leaves = tree.get_total_leafs()  # Количество листьев в дереве
            self.trees.append((tree, cols_idx, lr, num_leaves))  # Сохраняем дерево, признаки, lr, листья

            # Регуляризация: добавляем вклад деревьев, умноженный на reg
            regularization_term = self.reg * sum(leaf_count for _, _, _, leaf_count in self.trees)

            # Обновление предсказаний с учетом регуляризации
            current_prediction[rows_idx] += lr * tree.predict(X_sample) + regularization_term
            
             # Суммируем важность признаков
            feature_importance = tree.feature_importances()
            for feature, importance in feature_importance.items():
                self.fi[feature] += importance

            # Вывод метрик
            if verbose and i % verbose == 0:
                loss_value = self._calculate_loss(y, current_prediction)
                print(f"{i}. Loss[{self.loss}]: {loss_value:.2f}")
                
            # Ранняя остановка
            if X_eval is not None and y_eval is not None and early_stopping is not None:
                eval_pred = self.predict(X_eval)
                score = self._calculate_metric(y_eval, eval_pred)

                # Если это первый запуск или метрика улучшилась
                if best_score is None or score < best_score:
                    best_score = score
                    best_iter = i
                else:
                    # Если метрика не улучшилась за early_stopping итераций, остановим
                    if i - best_iter >= early_stopping:
                        print(f"Early stopping at iteration {i} with best score: {best_score:.4f}")
                        break

        # После завершения обучения сохраняем финальное значение метрики, если задана
        if self.metric:
            self.best_score = self._calculate_metric(y, current_prediction)
        else:
            self.best_score = self._calculate_loss(y, current_prediction)
                
    def _build_tree(self, X: pd.DataFrame, residual: pd.Series):
        # Создание экземпляра дерева регрессии
        tree = MyForestReg(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            max_leafs=self.max_leafs,
            bins=self.bins
        )
        # Обучение дерева на остатках
        tree.fit(X, residual)
        return tree

    # Метод для подсчета общего числа листьев
    def get_total_leafs(self):
        # Суммируем количество листьев для каждого дерева в лесу
        return sum(tree.get_total_leafs() for tree in self.trees)
    
    def feature_importances(self):
        # Метод для получения важности признаков
        return self.fi
    
     # Метод предсказания
    def predict(self, X: pd.DataFrame):
        # Начальное предсказание
        total_prediction = np.full(X.shape[0], self.pred_0)  # Массив с начальным предсказанием для всех строк

        for i, (tree, cols, lr, _) in enumerate(self.trees, start=1):  # Добавлен `_` для пропуска количества листьев
            # Если learning_rate был функцией, пересчитаем для текущей итерации
            if callable(self.learning_rate):
                lr = self.learning_rate(i)
            # Регуляризация при предсказании
            regularization_term = self.reg * sum(leaf_count for _, _, _, leaf_count in self.trees[:i])
            total_prediction += lr * tree.predict(X[cols]) + regularization_term
        return total_prediction


In [63]:
#Тест остановки
X, y = make_regression(n_samples=50, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(5)])
y = pd.Series(y)

# Обучение модели с ранней остановкой
model = MyBoostReg(
    n_estimators=50, 
    learning_rate=0.1, 
    max_depth=3, 
    reg=0.1, 
    metric="MSE"  # Используем метрику "MSE", определённую в классе
)

# Обучение с использованием всего набора данных для валидации
model.fit(X, y, X_eval=X, y_eval=y, early_stopping=5)

# Предсказание
predictions = model.predict(X)

# Вывод результата
print(f"Best score on evaluation set: {model.best_score:.4f}")

Early stopping at iteration 7 with best score: 14701.4875
Best score on evaluation set: 18368.6183


In [51]:
# Тест важности фичей
X, y = make_regression(n_samples=20, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(5)])
y = pd.Series(y)  # Преобразуем y в Series

# Обучение модели
model = MyBoostReg(n_estimators=10, learning_rate=0.1, max_depth=3, reg=0.1)
model.fit(X, y)

# Вывод важности признаков
print("Feature Importances:")
for feature, importance in model.feature_importances().items():
    print(f"{feature}: {importance:.4f}")


Feature Importances:
feature_0: 19021.6635
feature_1: 0.0000
feature_2: 314094.7646
feature_3: 95588.6914
feature_4: 18397.4976


In [40]:
#Тест регуляризации
X, y = make_regression(n_samples=20, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y)

# Тестируем с разными значениями reg
for reg_value in [0.0, 0.1, 0.5]:
    print(f"\nTesting with reg={reg_value}")
    model = MyBoostReg(
        n_estimators=5, 
        learning_rate=0.1, 
        max_depth=3, 
        loss="MSE", 
        metric="R2", 
        reg=reg_value
    )
    model.fit(X, y)
    print(f"Best score with reg={reg_value}: {model.best_score:.4f}")


Testing with reg=0.0
Best score with reg=0.0: 0.1612

Testing with reg=0.1
Best score with reg=0.1: 0.1479

Testing with reg=0.5
Best score with reg=0.5: -0.0569


In [35]:
# тест динамического learning rate
X, y = make_regression(n_samples=20, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y)

# Learning rate как число
model_static_lr = MyBoostReg(n_estimators=5, learning_rate=0.1, max_depth=3, loss="MSE", metric="R2")
model_static_lr.fit(X, y)
print("Predictions (static LR):", model_static_lr.predict(X).sum())

# Learning rate как функция
dynamic_lr = lambda iter: 0.5 * (0.85 ** iter)
model_dynamic_lr1 = MyBoostReg(n_estimators=5, learning_rate=dynamic_lr, max_depth=3, loss="MSE", metric="R2")
model_dynamic_lr1.fit(X, y)
print("Predictions (dynamic LR):", model_dynamic_lr1.predict(X).sum())

model_dynamic_lr2 = MyBoostReg(n_estimators=5, learning_rate=dynamic_lr, max_depth=2, loss="MSE", metric="MAPE")
model_dynamic_lr2.fit(X, y)
print("Predictions (dynamic LR):", model_dynamic_lr2.predict(X).sum())

Predictions (static LR): -680.1626942137246
Predictions (dynamic LR): -531.8174657234895
Predictions (dynamic LR): -531.8174657234895


In [ ]:
# Тест стохастического градиентного бустинга
X, y = make_regression(n_samples=20, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y)

# Тест модели с метрикой R2
model_with_r2 = MyBoostReg(n_estimators=5, learning_rate=0.1, max_depth=3, loss="MSE", metric="R2")
model_with_r2.fit(X, y)  # Без промежуточного лога
print(f"Best {model_with_r2.metric}: {model_with_r2.best_score:.2f}")

# Тест модели с метрикой RMSE
model_with_RMSE = MyBoostReg(n_estimators=5, learning_rate=0.1, max_depth=3, loss="MSE", metric="RMSE")
model_with_RMSE.fit(X, y)  # Без промежуточного лога
print(f"Best {model_with_RMSE.metric}: {model_with_RMSE.best_score:.2f}")

Best R2: 0.16
Best RMSE: 185.53


In [26]:
# Тест метрик
X, y = make_regression(n_samples=20, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y)

# Тест модели с метрикой R2
model_with_r2 = MyBoostReg(n_estimators=5, learning_rate=0.1, max_depth=3, loss="MSE", metric="R2")
model_with_r2.fit(X, y)  # Без промежуточного лога
print(f"Best {model_with_r2.metric}: {model_with_r2.best_score:.2f}")

# Тест модели с метрикой RMSE
model_with_RMSE = MyBoostReg(n_estimators=5, learning_rate=0.1, max_depth=3, loss="MSE", metric="RMSE")
model_with_RMSE.fit(X, y)  # Без промежуточного лога
print(f"Best {model_with_RMSE.metric}: {model_with_RMSE.best_score:.2f}")


Best R2: 0.38
Best RMSE: 160.03


In [24]:
# Тест градиента
X, y = make_regression(n_samples=20, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y)

# Создаем модели
model1 = MyBoostReg(n_estimators=10, learning_rate=0.1, max_depth=3, loss="MSE")
model2 = MyBoostReg(n_estimators=8, learning_rate=0.1, max_depth=3, loss="MAE")

# Обучаем модель с verbose=2
model1.fit(X, y, verbose=2)
model2.fit(X, y, verbose=3)

0. Loss[MSE]: 41038.85
2. Loss[MSE]: 33821.36
4. Loss[MSE]: 28105.41
6. Loss[MSE]: 23413.61
8. Loss[MSE]: 19901.45
10. Loss[MSE]: 17161.91
0. Loss[MAE]: 158.01
3. Loss[MAE]: 157.87
6. Loss[MAE]: 157.73


In [ ]:
#Тестирование предсказания
# Создаем первый небольшой набор данных
X1, y1 = make_regression(n_samples=20, n_features=5, noise=0.1, random_state=0)
X1 = pd.DataFrame(X1, columns=[f"feature_{i}" for i in range(X1.shape[1])])
y1 = pd.Series(y1)

# Создаем второй небольшой набор данных
X2, y2 = make_regression(n_samples=15, n_features=5, noise=0.2, random_state=1)
X2 = pd.DataFrame(X2, columns=[f"feature_{i}" for i in range(X2.shape[1])])
y2 = pd.Series(y2)

# Параметры модели
params = {
    'n_estimators': 5,
    'learning_rate': 0.1,
    'max_depth': 3,
    'min_samples_split': 2,
    'max_leafs': 5,
    'bins': None
}

# Создаем модель
model = MyBoostReg(**params)

# Тестирование на первом наборе данных
print("\nТестирование на первом наборе данных:")
model.fit(X1, y1)  # Обучаем модель
predictions1 = model.predict(X1)  # Делаем предсказания
total_prediction1 = predictions1.sum()  # Сумма предсказаний
print(f"Сумма предсказаний для первого набора данных: {total_prediction1:.2f}")

# Тестирование на втором наборе данных
print("\nТестирование на втором наборе данных:")
model.fit(X2, y2)  # Обучаем модель
predictions2 = model.predict(X2)  # Делаем предсказания
total_prediction2 = predictions2.sum()  # Сумма предсказаний
print(f"Сумма предсказаний для второго набора данных: {total_prediction2:.2f}")


Тестирование на первом наборе данных:
Сумма предсказаний для первого набора данных: 414.24

Тестирование на втором наборе данных:
Сумма предсказаний для второго набора данных: -103.45


In [ ]:
# Тест обучения
X, y = make_regression(n_samples=200, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y)

# Параметры для тестирования
test_params = [
    {"n_estimators": 3, "learning_rate": 0.1, "max_depth": 5, "min_samples_split": 2, "max_leafs": 10, "bins": 16},
    {"n_estimators": 5, "learning_rate": 0.05, "max_depth": 6, "min_samples_split": 3, "max_leafs": 12, "bins": None},
    {"n_estimators": 7, "learning_rate": 0.01, "max_depth": 8, "min_samples_split": 4, "max_leafs": 15, "bins": 8},
]

# Функция для подсчёта общего числа листьев во всех деревьях
def count_total_leaves(model):
    # Суммируем количество листьев всех деревьев в модели
    return sum(tree.get_total_leafs() for tree in model.trees)

# Тестирование
for params in test_params:
    print(f"\nТестирование с параметрами: {params}")
    
    # Создаём модель с текущими параметрами
    model = MyBoostReg(**params)
    
    # Обучаем модель
    model.fit(X, y)
    
    # Вывод результата
    print(f"pred_0: {model.pred_0:.2f}")
    total_leaves = count_total_leaves(model)  # Подсчитываем количество листьев
    print(f"Общее количество листьев: {total_leaves}")



Тестирование с параметрами: {'n_estimators': 3, 'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 2, 'max_leafs': 10, 'bins': 16}
pred_0: 23.30
Общее количество листьев: 362

Тестирование с параметрами: {'n_estimators': 5, 'learning_rate': 0.05, 'max_depth': 6, 'min_samples_split': 3, 'max_leafs': 12, 'bins': None}
pred_0: 23.30
Общее количество листьев: 724

Тестирование с параметрами: {'n_estimators': 7, 'learning_rate': 0.01, 'max_depth': 8, 'min_samples_split': 4, 'max_leafs': 15, 'bins': 8}
pred_0: 23.30
Общее количество листьев: 1169


In [6]:
# Тестирование класса
model1 = MyBoostReg()
model2 = MyBoostReg(n_estimators=10,  max_depth=5, min_samples_split=5, max_leafs=10, bins=6)
model3 = MyBoostReg(n_estimators=8, max_depth=5, min_samples_split=5, max_leafs=20, bins=16)

# Проверка
print(model1)
print(model2)
print(model3)


MyForestReg class: n_estimators=10, learning_rate=0.1, max_depth=5, min_samples_split=2, max_leafs=20, bins=None, pred_0=None, trees=[]
MyForestReg class: n_estimators=10, learning_rate=0.1, max_depth=5, min_samples_split=5, max_leafs=10, bins=6, pred_0=None, trees=[]
MyForestReg class: n_estimators=8, learning_rate=0.1, max_depth=5, min_samples_split=5, max_leafs=20, bins=16, pred_0=None, trees=[]
